# imports and data

In [64]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
import cv2
import keras
import os
from keras.callbacks import TensorBoard

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.layers.merge import Add, Concatenate

from PIL import Image

import keras
print(keras.__version__)

2.1.1


In [3]:
vgg_base = VGG16(weights='imagenet', include_top=False)

In [4]:
vgg_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [22]:
np.random.seed(1234)
from scipy.misc import imresize
from skimage.transform import resize

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
#         a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
#         b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
#         c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         imgs.append(np.dstack((a, b, c)))
#         imgs.append(np.dstack((band_1, band_2, band_3)))
#         img_orig = Image.fromarray(np.dstack((band_1, band_2, band_3)), 'RGB')
#         img = img_orig.resize((224, 224), img_orig)
        img = resize(np.dstack((band_1, band_2, band_3)), (224, 224), mode='constant')
        imgs.append(img)

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [23]:
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]

# remove all 3 below if you just want orginal data
Xtrain = get_more_images(Xtrain)
Xinc = np.concatenate((Xinc,Xinc,Xinc))
Ytrain = np.concatenate((Ytrain,Ytrain,Ytrain))

In [24]:
Xtrain.shape, Xinc.shape, Ytrain.shape

((4413, 224, 224, 3), (4413,), (4413,))

In [25]:
#split data for validation - bit of an arbitary split here

Xvalid = Xtrain[0:700]
Xinc_valid = Xinc[0:700]
Yvalid = Ytrain[0:700]

Xtrain_train = Xtrain[700:4413]
Xinc_train = Xinc[700:4413]
Ytrain_train = Ytrain[700:4413]

In [26]:
Xtrain_train.shape, Xinc_train.shape, Ytrain_train.shape, Xvalid.shape, Xinc_valid.shape, Yvalid.shape

((3713, 224, 224, 3), (3713,), (3713,), (700, 224, 224, 3), (700,), (700,))

In [27]:
def create_precomputed_data(model, data, data_labels):
    conv_features = model.predict(data)
    labels_onehot = to_categorical(data_labels)
    labels = data_labels
    return (conv_features, labels_onehot, labels)

In [51]:
# trn_conv_features, trn_labels, trn_labels_1 = create_precomputed_data(vgg_base, Xtrain_train, Ytrain_train)
# val_conv_features, val_labels, val_labels_1 = create_precomputed_data(vgg_base, Xvalid, Yvalid)
all_conv_features, all_labels, all_labels_1 = create_precomputed_data(vgg_base, Xtrain, Ytrain)

In [46]:
trn_conv_features.shape, trn_labels.shape, val_conv_features.shape, val_labels.shape

((3713, 7, 7, 512), (3713, 2), (700, 7, 7, 512), (700, 2))

In [41]:
assert trn_conv_features.shape == (3713, 7, 7, 512), "trn_conv_features not as expected"
assert trn_labels.shape == (3713, 2), "trn_labels not as expected"

assert val_conv_features.shape == (700, 7, 7, 512), "val_conv_features not as expected"
assert val_labels.shape == (700, 2), "val_labels not as expected"

In [62]:
RESULTS_DIR = '/home/ubuntu/data/iceberg/results/'

In [43]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def save_precomputed_data(conv_feats, labels, inc_angle, features_base_name="VGG16_conv_feats_aug/trn_"):
    save_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat', conv_feats)
    save_array(RESULTS_DIR+"/"+features_base_name+'labels.dat', np.array(labels))
    save_array(RESULTS_DIR+"/"+features_base_name+'inc_angle_feats.dat', conv_feats)
    
save_precomputed_data(trn_conv_features, trn_labels, Xinc_train, "VGG16_conv_feats_aug/trn_")
save_precomputed_data(val_conv_features, val_labels, Xinc_valid, "VGG16_conv_feats_aug/val_")

In [45]:
import bcolz
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="VGG16_conv_feats_aug/trn_"):
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return conv_feats, labels

trn_conv_features, trn_labels = load_precomputed_data("VGG16_conv_feats_aug/trn_")
val_conv_features, val_labels = load_precomputed_data("VGG16_conv_feats_aug/val_")

# simple model 

In [47]:
p = 0.3 

classifier_input_shape = (7, 7, 512)
classifier_input = Input(shape=classifier_input_shape)

x= Flatten()(classifier_input)
x = Dense(512, activation='relu')(x)
x = Dropout(p)(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(p)(x)
x = Dense(2, activation='softmax')(x)
                                                     
classifier_model_v1 = Model(classifier_input, x)

classifier_model_v1.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/iceberg/tb_logs_ice/', histogram_freq=0, write_graph=True, write_images=True)

In [49]:
classifier_model_v1.fit(trn_conv_features, trn_labels,
                                          batch_size=32, 
                                          epochs=50,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True, 
                                          callbacks=[tbCallBack])

Train on 3713 samples, validate on 700 samples
Epoch 1/50
3713/3713 [==============================] - 3s 805us/step - loss: 0.7241 - acc: 0.5834 - val_loss: 0.5659 - val_acc: 0.6786
Epoch 2/50
3713/3713 [==============================] - 3s 674us/step - loss: 0.4996 - acc: 0.7460 - val_loss: 0.3908 - val_acc: 0.8243
Epoch 3/50
3713/3713 [==============================] - 3s 674us/step - loss: 0.4012 - acc: 0.8093 - val_loss: 0.4526 - val_acc: 0.7714
Epoch 4/50
3713/3713 [==============================] - 2s 673us/step - loss: 0.3660 - acc: 0.8263 - val_loss: 0.4607 - val_acc: 0.8057
Epoch 5/50
3713/3713 [==============================] - 2s 671us/step - loss: 0.3265 - acc: 0.8519 - val_loss: 0.3271 - val_acc: 0.8529
Epoch 6/50
3713/3713 [==============================] - 2s 670us/step - loss: 0.2832 - acc: 0.8745 - val_loss: 0.3140 - val_acc: 0.8671
Epoch 7/50
3713/3713 [==============================] - 3s 679us/step - loss: 0.2585 - acc: 0.8898 - val_loss: 0.3199 - val_acc: 0.8557
E

# deeper model 

In [60]:
nf = 128
p = 0.2 # adding any dropout at all means it doesnt train at all

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='sigmoid', padding='same')(x)
x = BatchNormalization(axis=1)(x)

# x = MaxPooling2D((1,2))(x)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)

classifier_model_v2 = Model(classifier_input, x)

classifier_model_v2.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [61]:
classifier_model_v2.fit(trn_conv_features, trn_labels,
                                          batch_size=32, 
                                          epochs=50,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True, 
                                          callbacks=[tbCallBack])

Train on 3713 samples, validate on 700 samples
Epoch 1/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.5290 - acc: 0.7576 - val_loss: 0.4810 - val_acc: 0.7657
Epoch 2/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.3236 - acc: 0.8605 - val_loss: 0.3179 - val_acc: 0.8571
Epoch 3/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2610 - acc: 0.8901 - val_loss: 0.3771 - val_acc: 0.8429
Epoch 4/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2058 - acc: 0.9127 - val_loss: 0.6927 - val_acc: 0.7843
Epoch 5/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1615 - acc: 0.9372 - val_loss: 0.3214 - val_acc: 0.8743
Epoch 6/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1283 - acc: 0.9461 - val_loss: 0.3556 - val_acc: 0.8743
Epoch 7/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.0866 - acc: 0.9685 - val_loss: 0.3779 - val_acc: 0.8657
Epoch 8/

In [57]:
nf = 128
p = 0. # adding any dropout at all means it doesnt train at all

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)

# x = MaxPooling2D((1,2))(x)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)

classifier_model_v3 = Model(classifier_input, x)

classifier_model_v3.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [58]:
classifier_model_v3.fit(trn_conv_features, trn_labels,
                                          batch_size=32, 
                                          epochs=50,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True, 
                                          callbacks=[tbCallBack])

Train on 3713 samples, validate on 700 samples
Epoch 1/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.5313 - acc: 0.7498 - val_loss: 0.7826 - val_acc: 0.6957
Epoch 2/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.3282 - acc: 0.8529 - val_loss: 0.4505 - val_acc: 0.8071
Epoch 3/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2591 - acc: 0.8942 - val_loss: 0.2983 - val_acc: 0.8800
Epoch 4/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2205 - acc: 0.9076 - val_loss: 0.4919 - val_acc: 0.7986
Epoch 5/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1512 - acc: 0.9405 - val_loss: 1.0548 - val_acc: 0.6986
Epoch 6/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1279 - acc: 0.9534 - val_loss: 0.4099 - val_acc: 0.8771
Epoch 7/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.0948 - acc: 0.9623 - val_loss: 0.4143 - val_acc: 0.8600
Epoch 8/

# deeper model with inc angle 

In [95]:
inc_angle_input = Input(shape=(1,))

nf = 128
p = 0.2 

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)

# x = MaxPooling2D((1,2))(x)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

# x = GlobalAveragePooling2D()(x)
# x = Activation('softmax')(x)


x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

optimizer = Adam(lr=0.001, decay=0.0)
model_inc_angle = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model_inc_angle.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
# model_inc_angle.fit([trn_conv_features, Xinc], trn_labels, batch_size=batch_size, epochs=50, verbose=1,shuffle=True,
#           callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack], validation_split=0.25)


model_inc_angle.fit([trn_conv_features, Xinc_train], trn_labels,
                                          batch_size=32, 
                                          epochs=50,
                                          validation_data=([val_conv_features, Xinc_valid], val_labels),
                                          shuffle=True, 
                                          callbacks=[tbCallBack])

Train on 3713 samples, validate on 700 samples
Epoch 1/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.5566 - acc: 0.7106 - val_loss: 2.3891 - val_acc: 0.5071
Epoch 2/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.4129 - acc: 0.8139 - val_loss: 0.3530 - val_acc: 0.8479
Epoch 3/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.3411 - acc: 0.8519 - val_loss: 0.3392 - val_acc: 0.8529
Epoch 4/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2604 - acc: 0.8892 - val_loss: 0.3031 - val_acc: 0.8764
Epoch 5/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2404 - acc: 0.8964 - val_loss: 0.7549 - val_acc: 0.7636
Epoch 6/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1624 - acc: 0.9378 - val_loss: 0.3493 - val_acc: 0.8871
Epoch 7/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1428 - acc: 0.9438 - val_loss: 0.6258 - val_acc: 0.8264
Epoch 8/

In [71]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')

In [72]:
model_inc_angle.fit([all_conv_features, Xinc], all_labels,
                                          batch_size=batch_size, 
                                          epochs=50,
                                          validation_split=0.25,
                                          verbose=1,
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 3530 samples, validate on 883 samples
Epoch 1/50
3530/3530 [==============================] - 11s 3ms/step - loss: 0.1402 - acc: 0.9780 - val_loss: 0.0107 - val_acc: 1.0000
Epoch 2/50
3530/3530 [==============================] - 9s 3ms/step - loss: 0.0572 - acc: 0.9861 - val_loss: 0.0367 - val_acc: 0.9909
Epoch 3/50
3530/3530 [==============================] - 9s 3ms/step - loss: 0.0342 - acc: 0.9911 - val_loss: 0.0041 - val_acc: 0.9989
Epoch 4/50
3530/3530 [==============================] - 9s 3ms/step - loss: 0.0249 - acc: 0.9938 - val_loss: 0.0128 - val_acc: 0.9977
Epoch 5/50
3530/3530 [==============================] - 9s 3ms/step - loss: 0.0152 - acc: 0.9966 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 6/50
3530/3530 [==============================] - 9s 3ms/step - loss: 0.0114 - acc: 0.9980 - val_loss: 7.2979e-04 - val_acc: 1.0000
Epoch 7/50
3530/3530 [==============================] - 9s 3ms/step - loss: 0.0083 - acc: 0.9983 - val_loss: 5.9467e-04 - val_acc: 1.0000
Epoch 

In [75]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model_inc_angle.evaluate([all_conv_features, Xinc], all_labels, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

4413/4413 [==============================] - 7s 2ms/step
Train score: 0.00213907369083
Train accuracy: 0.999773396782


In [78]:
data_dir = '/home/ubuntu/data/iceberg'

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle


In [79]:
Xtest.shape, Xtest_inc.shape

((8424, 224, 224, 3), (8424,))

In [81]:
test_conv_features = vgg_base.predict(Xtest)

In [82]:
pred_test = model_inc_angle.predict([test_conv_features, Xtest_inc])

In [83]:
pred_test[1,1]

0.99957806

In [92]:
# this is the right way round
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('cnn_train_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  3.952795e-01
1  4023181e  9.995781e-01
2  b20200e4  9.970840e-01
3  e7f018bb  9.993762e-01
4  4371c8c3  9.994198e-01
5  a8d9b1fd  9.996321e-01
6  29e7727e  1.797852e-32
7  92a51ffb  9.995446e-01
8  c769ac97  1.552835e-15
9  aee0547d  1.769691e-29


In [93]:
os.getcwd()


'/home/ubuntu/git/learningWithKaggle/ice'

In [96]:
np.around(score[1])

1.0